In [1]:
# import kbase
import os
# local_cobrakbase_path = 'C:\\Users\\Andrew Freiburger\\Dropbox\\My PC (DESKTOP-M302P50)\\Documents\\UVic Civil Engineering\\Internships\\Agronne\\cobrakbase'
# os.environ["HOME"] = local_cobrakbase_path
import cobrakbase
token = 'TKWQA762H2SMAYRES3BFUP37LKGNGQSM'
kbase = cobrakbase.KBaseAPI(token)

ftp_path = '../../../ModelSEEDDatabase'

# import cFBA
%run ModelSEEDpy/modelseedpy/core/mscommunity.py
%run ModelSEEDpy/modelseedpy/core/fbahelper.py
%matplotlib inline

cobrakbase 0.2.8


## Define the model

In [2]:
modelInfo_2 = ["CMM_iAH991V2_iML1515.kb",40576]
mediaInfo_2 = ["Btheta_Ecoli_minimal_media",40576]

model = kbase.get_from_ws(modelInfo_2[0],modelInfo_2[1])
media = kbase.get_from_ws(mediaInfo_2[0],mediaInfo_2[1])

# drain_fluxes function

In [25]:
def drain_fluxes(predict_abundance = False):
    biomass_drains = {}
    # parse the metabolites in the biomass reactions
    for reaction in model.reactions:
        if re.search('^bio\d+$', reaction.id):
            for metabolite in reaction.metabolites:
                # identify the biomass metabolites
                msid = FBAHelper.modelseed_id_from_cobra_metabolite(metabolite)
                m = re.search('[a-z](\d+)', metabolite.compartment).group()
                if msid == "cpd11416" and m != None:
                    # evaluate only cross-feeding
                    index = m[1]
                    if index != "0" and index not in biomass_drains and predict_abundance:
                        print(f"Making biomass drain: {metabolite.id}")
                        drain_reaction = FBAHelper.add_drain_from_metabolite_id(model, metabolite.id,0,100,"DM_")
                        model.add_reactions([drain_reaction])
                        biomass_drains[index] = drain_reaction
                
    # print the objective value for each specie in the model
    for i in range(1,len(biomass_drains)+1):
        FBAHelper.set_objective_from_target_reaction(model,f"DM_cpd11416_c{i}")
        sol=model.optimize()
        print(f"species {i} objective value: {sol.objective_value}")
        
    return biomass_drains

# Test the function

In [26]:
drain_fluxes(predict_abundance = True)

Making biomass drain:cpd11416_c1
Making biomass drain:cpd11416_c2
species 1 objective value: 75.52804108181633
species 2 objective value: 86.91587102429742


{'1': <Reaction DM_cpd11416_c1 at 0x7f456c6c0b50>,
 '2': <Reaction DM_cpd11416_c2 at 0x7f456c6c0c40>}